# Semantic Segmentation with OpenVINO™ using Segmenter

Semantic segmentation is a difficult computer vision problem with many applications such as autonomous driving, robotics, augmented reality, and many others.
Its goal is to assign labels to each pixel according to the object it belongs to, creating so-called segmentation masks.
To properly assign this label, the model needs to consider the local as well as global context of the image.
This is where transformers offer their advantage as they work well in capturing global context.

Segmenter is based on Vision Transformer working as an encoder, and Mask Transformer working as a decoder.
With this configuration, it achieves good results on different datasets such as ADE20K, Pascal Context, and Cityscapes.
It works as shown in the diagram below, by taking the image, splitting it into patches, and then encoding these patches.
Mask transformer combines encoded patches with class masks and decodes them into a segmentation map as the output, where each pixel has a label assigned to it.


<div style="text-align:center">
    <img src="https://user-images.githubusercontent.com/24582831/148507554-87eb80bd-02c7-4c31-b102-c6141e231ec8.png" width="50%"/>
</div>

More about the model and its details can be found in the following paper:
[Segmenter: Transformer for Semantic Segmentation](https://arxiv.org/abs/2105.05633) or in the [repository](https://github.com/rstrudel/segmenter).

To demonstrate how to convert and use Segmenter in OpenVINO, this notebook consists of the following steps:

* Preparing PyTorch Segmenter model
* Preparing preprocessing and visualization function
* Validating inference of original model
* Converting PyTorch model to ONNX
* Converting ONNX to OpenVINO IR
* Validating inference of converted model

# Get and prepare PyTorch model

First thing we'll need to do is clone [repository](https://github.com/rstrudel/segmenter) containing model and helper functions. We will use Tiny model with mask transformer, that is Seg-T-Mask/16. There are also better, but much larger models available in linked repo. This model is pretrained on [ADE20K](https://groups.csail.mit.edu/vision/datasets/ADE20K/) dataset used for segmentation.

PyTorch models are usually instance of [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) class, initialized by a state dictionary containing model weights.
Typical steps to get model are therefore:

1. Create instance of model class
2. Load checkpoint state dict, which contains pretrained model weights
3. Turn model to evaluation for switching some operations to inference mode

In our case, the code from repository already contains functions that create model and load weights, but we will need to download config and trained wight (checkpoint) file.

## Prerequisites

In [ ]:
import sys
from pathlib import Path

import numpy as np
import yaml

sys.path.append("../utils")
from notebook_utils import download_file

In [ ]:
# we'll need timm and mmsegmentation module, to use segmenter repo
!pip install timm mmsegmentation einops mmcv -q

First, we will clone the Segmenter repo and then download weights and config for our model.

In [ ]:
# clone Segmenter repo
if not Path("segmenter").exists():
    !git clone https://github.com/rstrudel/segmenter
else:
    print("Segmenter repo already cloned")

%cd segmenter

In [ ]:
# download config and pretrained model weights
# here we use tiny model, there are also better but larger models available in repository
WEIGHTS_LINK = "https://www.rocq.inria.fr/cluster-willow/rstrudel/segmenter/checkpoints/ade20k/seg_tiny_mask/checkpoint.pth"
CONFIG_LINK = "https://www.rocq.inria.fr/cluster-willow/rstrudel/segmenter/checkpoints/ade20k/seg_tiny_mask/variant.yml"

MODEL_DIR = Path("../model/")
MODEL_DIR.mkdir(exist_ok=True)

download_file(WEIGHTS_LINK, directory=MODEL_DIR, show_progress=True)
download_file(CONFIG_LINK, directory=MODEL_DIR, show_progress=True)

WEIGHT_PATH = MODEL_DIR / "checkpoint.pth"
CONFIG_PATH = MODEL_DIR / "variant.yaml"

Define utility functions for preprocessing and visualizing the results.

In [ ]:
from PIL import Image
import torch
import torchvision.transforms.functional as F
from segm.data.utils import dataset_cat_description, seg_to_rgb
from segm.data.ade20k import ADE20K_CATS_PATH


def preprocess(im: Image, normalization: dict) -> torch.Tensor:
    """
    Preprocess image: scale, normalize and unsqueeze

    :param im: input image
    :param normalization: dictionary containing normalization data from config file
    :return:
            im: processed (scaled and normalized) image
    """
    # change PIL image to tensor and scale to [0, 1]
    im = F.pil_to_tensor(im).float() / 255
    # normalize by given mean and standard deviation
    im = F.normalize(im, normalization["mean"], normalization["std"])
    # change dim from [C, H, W] to [1, C, H, W]
    im = im.unsqueeze(0)

    return im


def visualise_mask(pil_im: Image, results: torch.Tensor) -> Image:
    """
    Visualize segmentation masks combined with the image

    :param pil_im: original input image
    :param results: tensor containing segmentation masks for each pixel
    :return:
            pil_blend: image with colored segmentation masks overlay
    """
    cat_names, cat_colors = dataset_cat_description(ADE20K_CATS_PATH)

    # 3D array, where each pixel has values for all classes, take index of max as label
    seg_map = results.argmax(0, keepdim=True)
    seg_rgb = seg_to_rgb(seg_map, cat_colors)
    seg_rgb = (255 * seg_rgb.cpu().numpy()).astype(np.uint8)
    pil_seg = Image.fromarray(seg_rgb[0])

    # overlay segmentation mask over original image
    pil_blend = Image.blend(pil_im, pil_seg, 0.5).convert("RGB")

    return pil_blend

## Loading PyTorch model

We will now use already provided helper functions from repository to initialize the model.

In [ ]:
from segm.model.factory import load_model

pytorch_model, config = load_model(WEIGHT_PATH)
# put model into eval mode, to set it for inference
pytorch_model.eval()

Load normalization settings from config file.

In [ ]:
from segm.data.utils import STATS

normalization_name = config["dataset_kwargs"]["normalization"]
normalization = STATS[normalization_name]

## Check inference of original model

We will perform segmentation on example image `coco_hollywood.jpg`.

In [ ]:
from segm.model.utils import inference

# load image with PIL
pil_image = Image.open("../../data/image/coco_hollywood.jpg")

# preprocess image with normalization params loaded in cell above
image = preprocess(pil_image, normalization)

# inference function needs some meta parameters, where we specify that we don't flip images in inference mode
im_meta = dict(flip=False)
# perform inference with function from repository
original_results = inference(model=pytorch_model,
                             ims=[image],
                             ims_metas=[im_meta],
                             ori_shape=image.shape[2:4],
                             window_size=config["inference_kwargs"]["window_size"],
                             window_stride=config["inference_kwargs"]["window_stride"],
                             batch_size=1)

After inference is complete, we need to transform output to segmentation mask where each class has specified color.

In [ ]:
# combine segmentation mask with image
blended_image = visualise_mask(pil_image, original_results)

# show image with segmentation mask overlay
blended_image

We can see that model segments the image into meaningful parts.
Since we are using tiny variant of model, the result is not as good as it is with larger models, but it already shows nice segmentation performance.

## Export to ONNX

Now that we've verified that inference of PyTorch model works, we will first export it to ONNX format.

To do this, we first get input dimensions from model configuration file and create torch dummy input containing input dimensions.
After that we use export function from PyTorch to convert model to ONNX.
The process can generate some warnings, but they are not a problem.

In [ ]:
import torch.onnx

# get input sizes from config file
image_size = config["dataset_kwargs"]["image_size"]
batch_size = config["dataset_kwargs"]["batch_size"]
channels = 3

# make dummy input with correct shapes obtained from config file
dummy_input = torch.randn(batch_size, channels, image_size, image_size)

onnx_path = MODEL_DIR / "segmenter.onnx"

# export to onnx format
torch.onnx.export(pytorch_model,
                  dummy_input,
                  onnx_path,
                  input_names=['input'],                        # the model's input names
                  output_names=['output'],                      # the model's output names
                  dynamic_axes={'input': {0: 'batch_size'},     # variable length bach size
                                'output': {0: 'batch_size'}})

## Convert ONNX model to OpenVINO Intermediate Representation (IR)

While ONNX models are directly supported by OpenVINO runtime, it can be useful to convert them to IR format to take advantage of OpenVINO optimization tools and features.
 `mo.convert_model` function can be used for converting model using OpenVINO Model Optimizer.
 The function returns instance of OpenVINO Model class, which is ready to use in Python interface but can also be serialized to OpenVINO IR format for future execution.

In [ ]:
from openvino.tools import mo
from openvino.runtime import serialize

model = mo.convert_model(str(MODEL_DIR / "segmenter.onnx"))
# serialize model for saving IR
serialize(model, str(MODEL_DIR / "segmenter.xml"))

## Verify converted model inference

To test that model was successfully converted, we can use same inference function from original repository, but we need to make custom class.

`SegmenterOV` class contains OpenVINO model, with all attributes and methods required by inference function.
This way we don't need to write any additional custom code required to process input.

In [ ]:
from openvino.runtime import Core


class SegmenterOV:
    """
    Class containing OpenVINO model with all attributes required to work with inference function.

    :param model: compiled OpenVINO model
    :type model: CompiledModel
    :param output_blob: output blob used in inference
    :type output_blob: ConstOutput
    :param config: config file containing data about model and its requirements
    :type config: dict
    :param n_cls: number of classes to be predicted
    :type n_cls: int
    :param patch_size: size patches that images is split into
    :type patch_size: int
    :param normalization:
    :type normalization: dict

    """

    def __init__(self, model_path: Path):
        """
        Constructor method.
        Initializes OpenVINO model and sets all required attributes

        :param model_path: path to model's .xml file, also containing variant.yml
        """
        # init OpenVino core
        core = Core()
        # read model
        model_xml = core.read_model(model_path)
        self.model = core.compile_model(model_xml, 'CPU')
        self.output_blob = self.model.output(0)

        # load model configs
        variant_path = Path(model_path).parent / "variant.yml"
        with open(variant_path, "r") as f:
            self.config = yaml.load(f, Loader=yaml.FullLoader)

        # load normalization specs from config
        normalization_name = self.config["dataset_kwargs"]["normalization"]
        self.normalization = STATS[normalization_name]

        # load number of classes from config
        self.n_cls = self.config["net_kwargs"]["n_cls"]

        # load patch size from config
        self.patch_size = self.config["net_kwargs"]["patch_size"]

    def forward(self, data: torch.Tensor) -> torch.Tensor:
        """
        Perform inference on data and return the result in Tensor format

        :param data: input data to model
        :return: data inferred by model
        """
        return torch.from_numpy(self.model(data)[self.output_blob])

Now that we have created SegmenterOV helper class, we can use it in inference function.

In [ ]:
# load model into SegmenterOV class
model = SegmenterOV(MODEL_DIR / "segmenter.xml")

In [ ]:
# perform inference with same function as in case of PyTorch model from repository
results = inference(model=model,
                    ims=[image],
                    ims_metas=[im_meta],
                    ori_shape=image.shape[2:4],
                    window_size=model.config["inference_kwargs"]["window_size"],
                    window_stride=model.config["inference_kwargs"]["window_stride"],
                    batch_size=1)

In [ ]:
# combine segmentation mask with image
converted_blend = visualise_mask(pil_image, results)

# show image with segmentation mask overlay
converted_blend

As we can see, we get the same results as with original model.